In [1]:
!nvidia-smi

Thu Oct 20 07:28:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd drive/MyDrive/CatsvDogs/

/content/drive/MyDrive/CatsvDogs


In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
image_size = (244, 244)
batch_size = 16

In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "petimages",
    validation_split=0.2,
    labels='inferred',
    label_mode='binary',
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
)

Found 4000 files belonging to 2 classes.
Using 3200 files for training.


In [6]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "petimages",
    validation_split=0.2,
    subset="validation",
    labels='inferred',
    label_mode='binary',
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
)

Found 4000 files belonging to 2 classes.
Using 800 files for validation.


In [7]:
# data_augmentation = keras.Sequential(
#     [
#         layers.RandomFlip("horizontal"),
#         layers.RandomRotation(0.1),
#     ]
# )

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1/255.),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
metrics=[ 
    #tf.keras.metrics.BinaryAccuracy(name="accuracy", dtype=None, threshold=0.4),
    "accuracy",
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tf.keras.metrics.TrueNegatives(name="tn"),
    tf.keras.metrics.TruePositives(name="tp"),
    tf.keras.metrics.FalseNegatives(name="fn"),
    tf.keras.metrics.FalsePositives(name="fp")
]

In [14]:
# model.build(INPUT_SHAPE)

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=metrics
)

In [10]:
!nvidia-smi

Thu Oct 20 07:28:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |    314MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
model.build((None, 244, 244, 3))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 244, 244, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 242, 242, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 121, 121, 128)    0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 121, 121, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 119, 119, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 59, 64)       0         
 2D)                                                    

In [14]:
model.fit(
    train_ds,
    #steps_per_epoch=15,
    epochs = 10,
    validation_data=val_ds,
)

Epoch 1/10
200/200 [==============================] - 27s 105ms/step - loss: 0.7108 - accuracy: 0.5100 - precision: 0.5096 - recall: 0.5630 - tn: 730.0000 - tp: 902.0000 - fn: 700.0000 - fp: 868.0000 - val_loss: 0.6821 - val_accuracy: 0.6413 - val_precision: 0.6034 - val_recall: 0.8141 - val_tn: 189.0000 - val_tp: 324.0000 - val_fn: 74.0000 - val_fp: 213.0000
Epoch 2/10
200/200 [==============================] - 21s 102ms/step - loss: 0.6596 - accuracy: 0.6044 - precision: 0.5950 - recall: 0.6567 - tn: 882.0000 - tp: 1052.0000 - fn: 550.0000 - fp: 716.0000 - val_loss: 0.6577 - val_accuracy: 0.6925 - val_precision: 0.6784 - val_recall: 0.7261 - val_tn: 265.0000 - val_tp: 289.0000 - val_fn: 109.0000 - val_fp: 137.0000
Epoch 3/10
200/200 [==============================] - 21s 102ms/step - loss: 0.6169 - accuracy: 0.6603 - precision: 0.6562 - recall: 0.6754 - tn: 1031.0000 - tp: 1082.0000 - fn: 520.0000 - fp: 567.0000 - val_loss: 0.6446 - val_accuracy: 0.7050 - val_precision: 0.6808 - val_

In [15]:
loss, accuracy, precision, recall, tn, tp, fn, fp = model.evaluate(val_ds, verbose=1,batch_size=16)
print("loss - ", loss)
print("accuracy - ", accuracy)
print("precision - ", precision)
print("recall - ", recall)
print("TN - ", tn)
print("TP - ", tp)
print("FN - ", fn)
print("FP - ", fp)

50/50 [==============================] - 3s 44ms/step - loss: 0.5341 - accuracy: 0.7362 - precision: 0.7634 - recall: 0.6809 - tn: 318.0000 - tp: 271.0000 - fn: 127.0000 - fp: 84.0000
loss -  0.5341211557388306
accuracy -  0.7362499833106995
precision -  0.7633802890777588
recall -  0.6809045076370239
TN -  318.0
TP -  271.0
FN -  127.0
FP -  84.0
